In [11]:
import pandas as pd
import random as rd
import math as m
import sklearn.cluster as skc

In [2]:
def split(D):
    used = []
    test = []
    train = []
    while len(test) < len(D) * 0.2:
        i = rd.randint(0, len(D) - 1)
        if i not in used:
            used.append(i)
            test.append(D.iloc[i].values.tolist())
    for i in range(len(D)):
        if i not in used:
            train.append(D.iloc[i].values.tolist())
    return (train, test)

def dist_euc(p1, p2):
    dist = 0
    if len(p1) != len(p2):
        return 0
    else:
        for i in range(len(p1)):
            dist += (p1[i] - p2[i]) ** 2
        return m.sqrt(dist)

def dist_man(p1, p2):
    dist = 0
    if len(p1) != len(p2):
        return 0
    else:
        for i in range(len(p1)):
            dist += p1[i] + p2[i]
        return dist


def dist_cos(p1, p2):
    dist = 0
    if len(p1) != len(p2):
        return 0
    else:
        l = len(p1)
        dot = sum(i[0] * i[1] for i in zip(p1, p2))
        return dot / (dist_euc([0] * l, p1) * dist_euc([0] * l, p2))

train, test = split(pd.read_csv('iris.csv'))

Problem 6\
Consider the following algorithm. Run it on Iris.tab and Mnist datasets and report your results.\
    1) Divide your data into random train and test sets\
    2) Training: for a specific label, find the average of each feature. (In case of mnist, this is pixel wise).  give your averages. In case of mint, give this as 10 images, one for each label\
    3) For each data point in the training set, find the closest from the average templates. Use Manhattan, Euclidean and Cosine distances (note that higher cosine value means lower distance). Give the point that label. \
    4) Do the same for the test set\
    5) Give the confusion matrix for training and test sets.\

In [18]:
# Iris Algorithm
def P6(train, test, key, avg):
    for row in train:
        if avg[row[4]] == 0:
            avg[row[4]] = [row[0:4], 1]
        else:
            avg[row[4]][1] += 1
            for i in range(len(row) - 1):
                avg[row[4]][0][i] = avg[row[4]][0][i] + (row[i] - avg[row[4]][0][i]) / avg[row[4]][1]

    train_vals = []
    test_vals = []
    for val in avg.values():
        train_vals.append(find_min_dist(val[0], train))
        test_vals.append(find_min_dist(val[0], test))

    conf_train = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    conf_test  = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

    for i in range(len(avg)):
        for j in range(len(avg)):
            conf_train[i][key[train_vals[i][j]]] += 1
            conf_test[i][key[test_vals[i][j]]] += 1

    print(avg)
    print(train_vals)
    print(conf_train)
    print(test_vals)
    print(conf_test)

def find_min_dist(avg, data):
    dist = [m.inf, m.inf, m.inf]
    keys = [[], [], []]
    for i in data:
        d = dist_euc(avg, i[:-1])
        if d < dist[0]:
            dist[0] = d
            keys[0] = i[-1]

        d = dist_man(avg, i[:-1])
        if d < dist[1]:
            dist[1] = d
            keys[1] = i[-1]

        d = dist_cos(avg, i[:-1])
        if d < dist[2]:
            dist[2] = d
            keys[2] = i[-1]

    return keys


key_iris = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2,
            0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}
avg = {'Iris-setosa': 0, 'Iris-versicolor': 0, 'Iris-virginica': 0}
P6(train, test, key_iris, avg)


{'Iris-setosa': [[4.961538461538462, 3.382051282051282, 1.4538461538461536, 0.24102564102564103], 39], 'Iris-versicolor': [[5.939024390243902, 2.729268292682927, 4.270731707317074, 1.3121951219512196], 41], 'Iris-virginica': [[6.628205128205129, 2.9820512820512826, 5.582051282051283, 2.0410256410256418], 39]}
[['Iris-setosa', 'Iris-setosa', 'Iris-virginica'], ['Iris-versicolor', 'Iris-setosa', 'Iris-setosa'], ['Iris-virginica', 'Iris-setosa', 'Iris-setosa']]
[[2, 0, 1], [2, 1, 0], [2, 0, 1]]
[['Iris-setosa', 'Iris-setosa', 'Iris-virginica'], ['Iris-versicolor', 'Iris-setosa', 'Iris-setosa'], ['Iris-virginica', 'Iris-setosa', 'Iris-setosa']]
[[2, 0, 1], [2, 1, 0], [2, 0, 1]]


Problem 7\
Consider the following algorithm. Run it on Iris.tab and Mnist datasets and report your results.\
    1) Divide your data into random train and test sets\
    2) Training: k-means clustering for k=1 to k=15 (i.e., run your algorithm 15 times). DO NOT USE LABLES. Give the centroids. In case of mint, give this as k images, one for each label. (you may use a library function)\
    3) Label a cluster using the majority label found in the cluster. \
    4) For each data point in the test set, find the closest from the centroid Use Manhattan, Euclidean and Cosine distances (note that higher cosine value means lower distance). Give the point that label. \
    5) Give the confusion matrix for training and test sets.\
    6) Form the elbow map - using %correct as a measure. (Test set accuracy)\

In [31]:
from xml.etree.ElementPath import find


def max_index(lst):
    max = 0
    index = 0
    for i in range(len(lst)):
        if lst[i] > max:
            max = lst[i]
            index = i
    return index


def find_cluster(clusters, data):
    dist = [m.inf, m.inf, m.inf]
    labels = [0, 0, 0]
    for i in range(len(clusters)):
        d = dist_euc(clusters[i], data[:-1])
        if d < dist[0]:
            dist[0] = d
            labels[0] = data[-1]

        d = dist_man(clusters[i], data[:-1])
        if d < dist[1]:
            dist[1] = d
            labels[1] = data[-1]

        d = dist_cos(clusters[i], data[:-1])
        if d < dist[2]:
            dist[2] = d
            labels[2] = data[-1]

    return labels

def P7Iris(train, test, key):
    train_lab = [i[:-1] for i in train]
    for i in range(1, 16):
        cluster_count = {}
        kmeans = skc.KMeans(i).fit(train_lab)
        labels = kmeans.labels_
        for i in range(len(train)):
            if labels[i] not in cluster_count:
                cluster_count[labels[i]] = [0, 0, 0]
            cluster_count[labels[i]][key[train[i][-1]]] += 1
        for i in cluster_count.keys():
            cluster_count[i] = (cluster_count[i], key[max_index(cluster_count[i])])
        # print(cluster_count)
    
        clusters = kmeans.cluster_centers_

        acc_train = [[0, 0] for i in range(3)]
        acc_test = [[0, 0] for i in range(3)]

        for i in range(len(train)):
            lab = find_cluster(clusters, train[i])
            for j in range(len(lab)):
                if lab[j] == cluster_count[kmeans.labels_[i]][1]:
                    acc_train[j][0] += 1
                else:
                    acc_train[j][1] += 1

        for i in range(len(test)):
            lab = find_cluster(clusters, test[i])
            for j in range(len(lab)):
                if lab[j] == cluster_count[kmeans.labels_[i]][1]:
                    acc_test[j][0] += 1
                else:
                    acc_test[j][1] += 1
        print(acc_train)
        print(acc_test)

        # conf_train = [[0, 0, 0], [0, 0, 0], [0, 0, 0]] * 3
        # conf_test  = [[0, 0, 0], [0, 0, 0], [0, 0, 0]] * 3

P7Iris(train, test, key_iris)


[[119, 0], [119, 0], [119, 0]]
[[9, 21], [9, 21], [9, 21]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]
[[119, 0], [119, 0], [119, 0]]
[[10, 20], [10, 20], [10, 20]]


Problem 8\
Consider the following algorithm. Run it on Iris.tab and Mnist datasets and report your results.\
    1) Divide your data into random train and test sets\
    2) Using kNN with Manhattan, Euclidean and cosine distances classify the training and test sets. Do this for k=1, k=5, k=10. (you may use a library function)\
    3) Give the confusion matrix for training and test sets.\
    4) Form the elbow map - using %correct as a measure. (Test set accuracy)